# JS 如何通过事件循环机制（EventLoop）实现异步执行

JS 是单线程同步语言，这意味着在任何时候，JS 引擎中的调用栈只能处理一个任务。

JS 通过事件循环机制实现异步操作，使其在处理 I/O 密集型任务时能够非阻塞地执行。与多线程语言相比，JS 的单线程模型使代码编写更简单，同时减少了竞态条件和死锁问题的发生。

接下来先通过一张图全面认识 Chrome 的事件循环机制：

<img src='https://raw.githubusercontent.com/binghuis/assets/main/excalidraw/eventloop.avif' width='860px' />

调用栈是一个后进先出的栈结构，存在于 JS 引擎，用于管理 JS 执行上下文的执行顺序。

之前文章介绍过：

> 当 V8 执行全局、模块或函数代码时，在创建阶段会生成相应的执行上下文。执行上下文的状态组件「变量环境组件」和「词法环境组件」共同记录当前词法环境中的变量和函数。

当一个新的执行上下文被创建时，它会被压入调用栈，JS 引擎总是执行栈顶的上下文。当当前上下文的代码执行完毕后，它会从调用栈中弹出，控制权返回到上一个上下文。如果某个函数创建了闭包，尽管该函数的执行上下文在执行完毕后会从调用栈中弹出，但其词法环境仍会被闭包引用，因此不会被垃圾回收机制立即回收。

调用栈和执行上下文的结构如下图：

<img src='https://raw.githubusercontent.com/binghuis/assets/main/excalidraw/ec.avif' width='560px' />

调用栈一次只能处理一个任务，并且任务是同步执行的，但为什么我们仍然可以使用异步操作？

处理异步操作在于将异步任务的处理从主线程中分离出来，通过事件循环机制和任务队列来管理和执行这些异步任务。这样，主线程可以继续执行其他同步任务，而异步操作在完成后，其回调函数会被放入任务队列中。当调用栈为空时，事件循环机制会将这些回调函数从任务队列中取出，放入调用栈中执行。

Node.js 运行时的事件循环机制与浏览器的稍有不同，主要区别在宿主环境提供的 API。

为了更好的了解事件循环机制，先讲几个比较重要的概念：

## JS 运行时（runtime）

指的是 JS 代码执行时的环境，包括 JS 引擎和宿主环境（宿主 API）。

## 宿主环境

通常指的是浏览器、Node.js 等宏观的 JS 执行环境，而运行时则是 JS 代码实际执行的基本运行环境。虽然 JS 引擎本质上是宿主环境的一部分，但由于其功能相对独立，所以在描述时往往将 JS 引擎与宿主环境并列列举，以便更好地理解其在 JS 执行中的角色。

> 当你浏览社区的时候你会发现，在社区讨论中，很多人默认将宿主环境和运行时环境视为同一个概念。

不同的宿主环境提供了各自独特的**宿主 API** 比如：

- **浏览器 Web APIs**：提供了 HTML DOM、Promise、Timers（setTimeout、setInterval）、Web Storage API（localStorage、
  sessionStorage）、File API（Blob、File、FileList）等。
- **Node.js 核心模块**：提供了 `fs`、`http` ，定时器等 API。

网络进程和渲染进程都是浏览器的进程，在浏览器渲染机制部分会详细介绍它们。

https://www.youtube.com/watch?v=u1kqx6AenYw 5:22

它们 为 JS 提供了额外的能力，使开发者在不同环境能实现特定的功能。

## 任务队列

鼠标点击队列：优先级最高

同源队列

队列可以按任何顺序执行，任务必须按照队列顺序执行。

任务队列是存放任务（宏任务）和微任务的数据结构。

同源任务进相同队列，同源队队列任务按队列顺序执行。

- 任务：包含整体代码、setTimeout、setInterval、I/O 操作等。这些任务通常较大，会在事件循环的每一轮中被处理。
- 微任务：包含 Promise 的 then 和 catch 回调、MutationObserver 等。这些任务通常较小，会在当前任务完成后、下一个任务开始之前被处理。

## Event Loop


https://stackoverflow.com/questions/74182571/why-is-a-xhr-request-treated-as-a-macro-task-while-a-fetch-request-is-treated-as
https://www.jsv9000.app/


https://262.ecma-international.org/6.0/#sec-lexical-environments

https://html.spec.whatwg.org/multipage/webappapis.html#task-source

https://w3c.github.io/uievents/#event-types-list


```js
while (true) {
  const queue = getNextQueue();
  const task = queue.pop();
  execute(task);
  while (microtaskQueue.hasTasks()) doMicrotask();

  if (isRepaintTime()) {
    const animationTasks = animationQueue.copyTasks();

    for (task in animationmasks) doAnimationTask(task);
    repaint();
  }
}
```

requestAnimationFrame 既不是微任务也不是宏任务，它在渲染管道页面绘制前执行。
动画运行时添加的动画会在下一帧运行


#### 内存堆

JS 引擎通过调用栈和堆执行代码和管理内存。

- 调用栈：
- 内存堆：

在执行上下文里，基本类型的变量值存在环境记录里，引用类型和函数的值存在堆内存里，环境记录里存的是堆的内存地址。

相关概念我们已经了解了，下面是事件循环机制的详细过程：
